## Load *.trc files

Choose/insert a directory for the cell files to be searched in (e.g. F:\Marburg\single_colour_tracking\resting)

In [1]:
from pySPT.widgets import widgetTrackAnalysis
from pySPT.analysis import coverSlip
from pySPT.analysis import trajectoryStatistics
widget_track_analysis = widgetTrackAnalysis.WidgetTrackAnalysis()
cover_slip = coverSlip.CoverSlip()
trajectory_stats = trajectoryStatistics.TrajectoryStatistics()
display(widget_track_analysis.dir_box, widget_track_analysis.dir_button)
widget_track_analysis.dir_button.on_click(widget_track_analysis.open_dir)
widget_track_analysis.dir_box.observe(widget_track_analysis.change_dir_box)

Text(value='C:\\Users\\pcoffice37\\Documents\\testing_file_search', description='directory', placeholder='Type…

Button(description='browse', style=ButtonStyle(), tooltip='browse for directory')

Choose/insert the complete directory for the roi.log file which contains the cell sizes (e.g. F:\Marburg\single_colour_tracking\resting\roi.log)

In [2]:
display(widget_track_analysis.roi_box, widget_track_analysis.roi_button)
widget_track_analysis.roi_button.on_click(widget_track_analysis.open_roi)
widget_track_analysis.roi_box.observe(widget_track_analysis.change_roi_box)

Text(value='F:\\Marburg\\single_colour_tracking\\resting\\roi.log', description='roi', placeholder='Type somet…

Button(description='browse', style=ButtonStyle(), tooltip='browse for roi')

## Run Analysis

In [3]:
display(widget_track_analysis.run_button)
def run_analysis(event):
    widget_track_analysis.create_clear_output()
    widget_track_analysis.searchSubFolders(widget_track_analysis.dir_name)
    if widget_track_analysis.got_dir and widget_track_analysis.got_roi:
        cover_slip.roi_file = widget_track_analysis.roi_name
        cover_slip.cell_files = widget_track_analysis.file_names
        cover_slip.create_cells()   
    else:
        widget_track_analysis.warning_file()
    widget_track_analysis.cells = cover_slip.cells
widget_track_analysis.run_button.on_click(run_analysis)

## Filter for parameters

## Plot diffusion histogram

In [4]:
trajectory_stats.cell_trajectories = cover_slip.cell_trajectories
trajectory_stats.get_index()
trajectory_stats.create_init_filter_lst()

## Choose trajectory to plot

Choose a cell.

In [5]:
def dropdown(event):
    widget_track_analysis.cell_options = widget_track_analysis.get_cell_names(cover_slip.cells)
widget_track_analysis.run_button.on_click(dropdown)
display(widget_track_analysis.drop_down_cells)

Dropdown(description='Number:', options=(), value=None)

Choose a trajectory.

In [6]:
def dropdown(event):      
    for cell in cover_slip.cells:
        if cell.name == widget_track_analysis.drop_down_cells.value:
            target_cell = cover_slip.cells.index(cell)
            trajectory_indexes = [i+1 for i, _ in enumerate(cover_slip.cell_trajectories[target_cell])]
            widget_track_analysis.trajectory_options = trajectory_indexes
            widget_track_analysis.drop_down_trajectories.options = widget_track_analysis.trajectory_options
widget_track_analysis.drop_down_cells.observe(dropdown)
display(widget_track_analysis.drop_down_trajectories)

Dropdown(description='Number:', options=(), value=None)

In [7]:
def plot_particle(event):
    widget_track_analysis.create_clear_output()
    cover_slip.plot_trajectory(widget_track_analysis.drop_down_cells.value, widget_track_analysis.drop_down_trajectories.value)
display(widget_track_analysis.plot_button)
widget_track_analysis.plot_button.on_click(plot_particle)

Button(description='plot', style=ButtonStyle(), tooltip='plot chosen trajectory')

## Save

In [8]:
def save_analysis(event):
    from pySPT.widgets import hdf5
    h5 = hdf5.Hdf5()
    h5.create_h5_file()
    h5.groups()
    x = cover_slip.cell_trajectories[0][0]
    c = cover_slip.cells[0]
    h5.data_settings(x.dt, c.pixel_size, x.tau_threshold, x.fit_area, x.dof, x.D_min)
    from pySPT.analysis import trackAnalysis
    track_analysis = trackAnalysis.TrackAnalysis()
    track_analysis.save_diff(cover_slip.cell_trajectories[0])
    track_analysis.save_rossier(cover_slip.cell_trajectories[0])
    diff_info = track_analysis.diffusion_info
    rossier_info = track_analysis.rossier_info
    h5.data_diffusion_info(track_analysis.number_of_trajectories, diff_info[:,0], diff_info[:,1], diff_info[:,2], diff_info[:,3], diff_info[:,4], diff_info[:,5])
    h5.data_rossier_info(track_analysis.number_of_trajectories, rossier_info[:,0],  rossier_info[:,1],  rossier_info[:,2],  rossier_info[:,3],  rossier_info[:,4],  rossier_info[:,5],  rossier_info[:,6],  rossier_info[:,7],  rossier_info[:,8],  rossier_info[:,9], rossier_info[:,10])
display(widget_track_analysis.save_button)
widget_track_analysis.save_button.on_click(save_analysis)

Button(description='save', style=ButtonStyle(), tooltip='save entire analysis')